# Домашняя работа №3
### Собрать информацию о вакансиях на вводимую должность с сайтов hh.ru и/или Superjob и/или работа.ру. Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:
1. Наименование вакансии.
2. Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
3. Ссылку на саму вакансию.
4. Cайт, откуда собрана вакансия.
5. По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с всех сайтов. Общий результат можно вывести с помощью dataFrame через pandas, сохранить в json, либо csv.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import time
import pandas as pd

In [2]:
i = 0
url = 'https://spb.hh.ru/search/vacancy'

params = {
    # 'from': 'suggest_post',
    'text': 'Data science',
    'page': i,
    'items_on_page': 20,
    'customDomain': 1,
    'enable_snippets': 'true',
    'hhtmFrom': 'vacancy_search_list',
    'clusters': 'true',
    'ored_clusters': 'true',
    'enable_snippets': 'true'
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
}

response = requests.get(url=url, params=params, headers=headers)
soup = bs(response.text, 'html.parser')

In [3]:
# Посчитаем колличество страниц в выдаче
num_pages = int(soup.find_all('a', {'data-qa': 'pager-page'})[-1].find('span').text)
num_pages

44

In [4]:
# Будем собирать вакансии со всех страниц
vacancies_list = []
currency_dict = {
  'руб.': 'RUB',
  'USD': 'USD',
  'EUR': 'EUR',
  'KZT': 'KZT'
}

for i in range(num_pages):
  params = {
    # 'from': 'suggest_post',
    'text': 'Data science',
    'page': i,
    'items_on_page': 20,
    'customDomain': 1,
    'enable_snippets': 'true',
    'hhtmFrom': 'vacancy_search_list',
    'clusters': 'true',
    'ored_clusters': 'true',
    'enable_snippets': 'true'
  }

  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
  }

  response = requests.get(url=url, params=params, headers=headers)
  soup = bs(response.text, 'html.parser')

  vacancies = soup.find_all('div', {'class': 'serp-item'})
  print(i, " - ", len(vacancies))

  for vacancy in vacancies:
    min = None
    max = None
    currency = None
    #time.sleep(2) # Чтобы не забанили
    vacancy_name = vacancy.find('a', {'class': 'serp-item__title'}).getText()
    vacancy_link = vacancy.find('a', {'class': 'serp-item__title'})['href']
    vacancy_salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    vacancy_salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    company = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).getText().replace('\xa0', ' ')
    location = vacancy.find('div', {'data-qa': 'vacancy-serp__vacancy-address', 'class': "bloko-text"}).getText()

    site = 'https://hh.ru/'
    if vacancy_salary is not None:
      vacancy_salary = vacancy_salary.getText().replace('\u202f', '').split()
      currency = currency_dict[vacancy_salary[-1]]
      if vacancy_salary[0].isdigit():
          min = int(vacancy_salary[0])
          max = int(vacancy_salary[2])
      elif vacancy_salary[0] == 'от':
          min = int(vacancy_salary[1])
          max = None
      elif vacancy_salary[0] == 'до':
          min = None
          max = int(vacancy_salary[1]) 

    vacancy_dict = {
        'name': vacancy_name,
        'link': vacancy_link,
        'salary_min': min,
        'salary_max': max,
        'currency': currency,
        'site': site,
        'company': company,
        'location': location
        
    }

    vacancies_list.append(vacancy_dict)

0  -  20
1  -  20
2  -  20
3  -  20
4  -  20
5  -  20
6  -  20
7  -  20
8  -  20
9  -  20
10  -  20
11  -  20
12  -  20
13  -  20
14  -  20
15  -  20
16  -  20
17  -  20
18  -  20
19  -  20
20  -  20
21  -  20
22  -  20
23  -  20
24  -  20
25  -  20
26  -  20
27  -  20
28  -  20
29  -  20
30  -  20
31  -  20
32  -  20
33  -  20
34  -  20
35  -  20
36  -  20
37  -  20
38  -  20
39  -  20
40  -  20
41  -  20
42  -  20
43  -  3


In [5]:
len(vacancies_list)

863

In [8]:
pd.DataFrame(vacancies_list).head(50)                                 

,name,link,salary_min,salary_max,currency,site,company,location
0,Data scientist,https://spb.hh.ru/vacancy/69724262?from=vacanc...,150000.0,200000.0,RUB,https://hh.ru/,Астор,Москва
1,Аналитик данных / Data Analyst,https://spb.hh.ru/vacancy/69647954?from=vacanc...,60000.0,90000.0,RUB,https://hh.ru/,ООО Оптимакрос,Москва
2,Senior Data Scientist (Yepp),https://spb.hh.ru/vacancy/67716744?from=vacanc...,5000.0,8500.0,EUR,https://hh.ru/,FUNCORP,Кипр
3,Стажер Data Scientist (NLP),https://spb.hh.ru/vacancy/70516166?from=vacanc...,NaN,NaN,None,https://hh.ru/,Сбер. Data Science,Москва
4,Data Scientist,https://spb.hh.ru/vacancy/70580362?from=vacanc...,NaN,NaN,None,https://hh.ru/,ООО СимбирСофт,Санкт-Петербург
5,Middle Data Scientist,https://spb.hh.ru/vacancy/69713433?from=vacanc...,250000.0,NaN,RUB,https://hh.ru/,ООО НТехЛаб,Москва
6,Data Scientist,https://spb.hh.ru/vacancy/70426481?from=vacanc...,NaN,NaN,None,https://hh.ru/,ООО ИЦ АЙ-ТЕКО,"Москва, Деловой центр и еще 2"
7,Data Scientist,https://spb.hh.ru/vacancy/69651789?from=vacanc...,NaN,NaN,None,https://hh.ru/,Click,"Санкт-Петербург, Петроградская"
8,Data analyst,https://spb.hh.ru/vacancy/69140213?from=vacanc...,NaN,NaN,None,https://hh.ru/,Вкусно - и точка,Москва
9,Data Science Team Lead,https://spb.hh.ru/vacancy/70531364?from=vacanc...,NaN,NaN,None,https://hh.ru/,ООО Гудфокаст,"Москва, Калужская"


In [7]:
import json
with open('result.json', 'w') as fp:
    json.dump(vacancies_list, fp)